In [1]:
import os
import glob
import pandas as pd

In [2]:
# Define working directory
os.chdir(r"C:\Users\Cristina Bardan\Desktop\Repositories\FinalProject-Spotify\Original_data")

In [3]:
# Use glob to match the pattern ‘csv’
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

# Combine all files in the list and export as CSV
df = pd.concat([pd.read_csv(f) for f in all_filenames ])
df.reset_index(drop=True, inplace=True)

In [4]:
df2 = df.drop(['track', 'artist', 'uri'], axis=1)

In [5]:
#Validation of correlation between variables
import numpy as np 
df3=df2.corr()

In [6]:
# Sin chorus_hit
data = df2.values
X = df2[['danceability', 'energy', 'key', 'loudness','mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness','valence', 'tempo', 'duration_ms', 'time_signature', 'sections']]  
y = data[:, 15]
print(data.shape, X.shape, y.shape)
data

(41106, 16) (41106, 14) (41106,)


array([[ 0.578  ,  0.471  ,  4.     , ..., 30.88059, 13.     ,  1.     ],
       [ 0.704  ,  0.854  , 10.     , ..., 41.51106, 10.     ,  1.     ],
       [ 0.162  ,  0.836  ,  9.     , ..., 65.32887, 13.     ,  0.     ],
       ...,
       [ 0.562  ,  0.314  , 10.     , ..., 21.11763, 10.     ,  1.     ],
       [ 0.622  ,  0.781  ,  7.     , ..., 47.13558, 11.     ,  0.     ],
       [ 0.664  ,  0.739  ,  2.     , ..., 42.50341, 14.     ,  1.     ]])

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)

## Data Preprocessing

In [8]:
#Scale features (X) using MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler(feature_range=(0,1)).fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test) 

X_train_scaled

array([[0.78947368, 0.52283395, 0.81818182, ..., 0.05569473, 0.8       ,
        0.06508876],
       [0.54048583, 0.83094117, 0.81818182, ..., 0.03957652, 0.8       ,
        0.04142012],
       [0.5       , 0.66588373, 0.81818182, ..., 0.03176775, 0.8       ,
        0.03550296],
       ...,
       [0.57692308, 0.36777999, 0.81818182, ..., 0.09203629, 0.8       ,
        0.09467456],
       [0.41902834, 0.16070793, 0.63636364, ..., 0.0441886 , 0.8       ,
        0.04733728],
       [0.29554656, 0.24873856, 0.81818182, ..., 0.0506431 , 0.6       ,
        0.06508876]])

One-hot encode the labels

In [9]:
#One-hot encode output labels (y)
from tensorflow.keras.utils import to_categorical
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

y_train_categorical

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [1., 0.]], dtype=float32)

## Creating and defining our Deep Learning Model Architecture

In [10]:
#Create a sequential model
from tensorflow.keras.models import Sequential
model = Sequential() 

from tensorflow.keras.layers import Dense
number_inputs = 14  

#Create hidden layers
model.add(Dense(units=14,activation='relu', input_dim=number_inputs))
model.add(Dense(units=120,activation='relu'))
model.add(Dense(units=80,activation='relu'))


#Create output layer
number_classes = 2
model.add(Dense(units=number_classes, activation='softmax')) 

In [11]:
#Model Summary
model.summary() 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 14)                210       
_________________________________________________________________
dense_1 (Dense)              (None, 120)               1800      
_________________________________________________________________
dense_2 (Dense)              (None, 80)                9680      
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 162       
Total params: 11,852
Trainable params: 11,852
Non-trainable params: 0
_________________________________________________________________


In [12]:
#Compile the Model
import tensorflow as tf

opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

## Training the Model

In [13]:
#Training the Model
history = model.fit(X_train_scaled, y_train_categorical, epochs=500, batch_size=2000, shuffle=True, verbose=2)

Epoch 1/500
16/16 - 0s - loss: 0.6641 - accuracy: 0.6153
Epoch 2/500
16/16 - 0s - loss: 0.6227 - accuracy: 0.6445
Epoch 3/500
16/16 - 0s - loss: 0.5821 - accuracy: 0.6829
Epoch 4/500
16/16 - 0s - loss: 0.5524 - accuracy: 0.7095
Epoch 5/500
16/16 - 0s - loss: 0.5349 - accuracy: 0.7309
Epoch 6/500
16/16 - 0s - loss: 0.5241 - accuracy: 0.7382
Epoch 7/500
16/16 - 0s - loss: 0.5183 - accuracy: 0.7409
Epoch 8/500
16/16 - 0s - loss: 0.5139 - accuracy: 0.7435
Epoch 9/500
16/16 - 0s - loss: 0.5114 - accuracy: 0.7435
Epoch 10/500
16/16 - 0s - loss: 0.5089 - accuracy: 0.7453
Epoch 11/500
16/16 - 0s - loss: 0.5081 - accuracy: 0.7455
Epoch 12/500
16/16 - 0s - loss: 0.5060 - accuracy: 0.7472
Epoch 13/500
16/16 - 0s - loss: 0.5067 - accuracy: 0.7472
Epoch 14/500
16/16 - 0s - loss: 0.5036 - accuracy: 0.7484
Epoch 15/500
16/16 - 0s - loss: 0.5015 - accuracy: 0.7507
Epoch 16/500
16/16 - 0s - loss: 0.5012 - accuracy: 0.7490
Epoch 17/500
16/16 - 0s - loss: 0.5006 - accuracy: 0.7506
Epoch 18/500
16/16 - 0s

## Validation of the Model

In [14]:
#Evaluate the Model using the testing data
#Compare Model performace between training and testing data
model_loss_train, model_accuracy_train = model.evaluate(X_train_scaled, y_train_categorical, verbose=2)
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_categorical, verbose=2)

print(f"TRAINING DATA --> Loss: {model_loss_train}, Accuracy: {model_accuracy_train}")    
print(f"TESTING DATA --> Loss: {model_loss}, Accuracy: {model_accuracy}")

964/964 - 1s - loss: 0.4460 - accuracy: 0.7875
322/322 - 0s - loss: 0.4766 - accuracy: 0.7729
TRAINING DATA --> Loss: 0.446012943983078, Accuracy: 0.7875052690505981
TESTING DATA --> Loss: 0.4765716791152954, Accuracy: 0.7728909254074097


## Saving the Trained Model

In [16]:
# Save the model
model.save("../Models/h5/schorushit.h5")